In [1]:
#@title Install Dependencies
import subprocess

packages = []
pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2', 'gdown', 'onnxruntime']
print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['sudo','apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")
subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

print('Packages up to date.')
firsttry = True

Updating and installing system packages...
Updating and installing pip packages...
Defaulting to user installation because normal site-packages is not writeable
  Using cached ffmpeg-1.4-py3-none-any.whl
  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/e0/c1/236fe9621584f32ffd6c1eeba4005f1c2932d2e21ef901c321ae51b9219d/onnxruntime-1.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 30.3 MB/s eta 0:00:00a 0:00:01


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.16.0
    Uninstalling onnxruntime-1.16.0:
      Successfully uninstalled onnxruntime-1.16.0
Packages up to date.


In [3]:
#@title Clone Repositories
%cd hackathon
import os

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('content/')
#Credit to Redoverflow on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
!git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
#Rename to keep backwards compatibility with old variants of Disconnected
os.rename("Mangio-RVC-Tweaks", "Mangio-RVC-Fork")
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Mangio-RVC-Fork/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder


/home/vishnu/hackathon
Cloning into 'Mangio-RVC-Tweaks'...
remote: Enumerating objects: 3098, done.
remote: Total 3098 (delta 0), reused 0 (delta 0), pack-reused 3098
Receiving objects: 100% (3098/3098), 13.84 MiB | 8.78 MiB/s, done.
Resolving deltas: 100% (1935/1935), done.
Cloning into 'torchcrepe'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 442 (delta 253), reused 395 (delta 229), pack-reused 5
Receiving objects: 100% (442/442), 72.20 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (253/253), done.


In [4]:
os.chdir('Mangio-RVC-Fork')

now_dir = "Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

In [5]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)

Compatible GPU detected: NVIDIA GeForce GTX 1060 6GB


In [6]:
%cd ..

/home/vishnu/hackathon/content


In [7]:
#@title Download Pretrained Models
#Didn't ask.

#V1 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G32k.pth -d Mangio-RVC-Fork/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D32k.pth -d Mangio-RVC-Fork/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G40k.pth -d Mangio-RVC-Fork/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D40k.pth -d Mangio-RVC-Fork/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G48k.pth -d Mangio-RVC-Fork/pretrained -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D48k.pth -d Mangio-RVC-Fork/pretrained -o f0D48k.pth

#V2 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G32k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D32k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G40k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D40k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G48k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D48k.pth -d Mangio-RVC-Fork/pretrained_v2 -o f0D48k.pth

#RMVPE and Hubert
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d Mangio-RVC-Fork -o rmvpe.pt

#fp_16 Variant JSONs
!rm -rf Mangio-RVC-Fork/configs/32k.json
!rm -rf Mangio-RVC-Fork/configs/40k.json
!rm -rf Mangio-RVC-Fork/configs/48k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/32k.json -d Mangio-RVC-Fork/configs -o 32k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d Mangio-RVC-Fork/configs -o 40k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/48k.json -d Mangio-RVC-Fork/configs -o 48k.json

[#b1aaf1 67MiB/69MiB(97%) CN:4 DL:22MiB]]m
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b1aaf1|OK  |    19MiB/s|Mangio-RVC-Fork/pretrained/f0G32k.pth

Status Legend:
(OK):download completed.
[#062647 104MiB/104MiB(99%) CN:1 DL:26MiB]0m]
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
062647|OK  |    25MiB/s|Mangio-RVC-Fork/pretrained/f0D32k.pth

Status Legend:
(OK):download completed.
[#99ec62 65MiB/69MiB(94%) CN:4 DL:21MiB]]m
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
99ec62|OK  |    19MiB/s|Mangio-RVC-Fork/pretrained/f0G40k.pth

Status Legend:
(OK):download completed.
[#daf084 104MiB/104MiB(99%) CN:1 DL:20MiB]0m]
Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=============================================

In [8]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

In [9]:
#@title Set Training Variables
now_dir = "Mangio-RVC-Fork"
experiment_name = "v_voice" #@param {type:"string"}
path_to_training_folder = "dataset/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "48k" #@param ["32k", "40k", "48k"] {allow-input: false}
cpu_threads = 2 #@param {type:"integer"}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"integer"}
pitch_guidance = True #@param {type:"boolean"}

assert crepe_hop_length!=None, "You need to input something for crepe_hop_length, silly."
assert crepe_hop_length>0, "Hop length must be more than 0."
assert crepe_hop_length<=512, "Save frequency must be less than 512."

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

In [11]:
%cd hackathon/content

/home/vishnu/hackathon/content


In [12]:
#@title Load Dataset
#@markdown If it doesn't already exist, create a folder in your Google Drive named 'rvcDisconnected' and place your zip file there. This will look for the following ZIP file inside that 'rvcDisconnected' folder.
dataset = "data_vishnu.zip"  #@param {type:"string"}

#@markdown This loader will load datasets in a similar fashion to the So-Vits-SVC dataset loader. For best results, your dataset should be formatted as such:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───file1.wav
#@markdown     ├───...
#@markdown     └───file999.wav
#@markdown ```
#@markdown Audio filenames do not matter. All audio files should be in WAV format for best compatibility.

# TODO: Add something to convert non-WAVs to WAV

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = 'drive/MyDrive/rvcDisconnected/' + dataset
final_directory = 'dataset'
temp_directory = 'temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d "{temp_directory}" -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp "{temp_directory}/{name}" "{expDir}"
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


Archive:  drive/MyDrive/rvcDisconnected/data_vishnu.zip
  inflating: temp_dataset/data_vishnu/file1.wav  
  inflating: temp_dataset/data_vishnu/file2.wav  
  inflating: temp_dataset/data_vishnu/file3.wav  
  inflating: temp_dataset/data_vishnu/file4.wav  
  inflating: temp_dataset/data_vishnu/file5.wav  
  inflating: temp_dataset/data_vishnu/file6.wav  
  inflating: temp_dataset/data_vishnu/file7.wav  
  inflating: temp_dataset/data_vishnu/file8.wav  
  inflating: temp_dataset/data_vishnu/file9.wav  
Sanitizing...
Dataset Type: Single Speaker
Dataset imported.


In [24]:
%cd ..

/home/vishnu/hackathon/content


In [28]:
!pwd

/home/vishnu/hackathon/content


In [29]:

#@title Preprocessing
# Change the Experiment Name and the Path to Training Folder. You shouldn't need to change anything else.
import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs("%s/logs/%s" % (now_dir, experiment_name), exist_ok=True)

#%cd Mangio-RVC-Fork
cmd = "python3 Mangio-RVC-Fork/trainset_preprocess_pipeline_print.py \"%s\" %s %s \"%s/logs/%s\" 1" % (pttf, sr, cpu_threads, now_dir, experiment_name)
print(cmd)
!$cmd

python3 Mangio-RVC-Fork/trainset_preprocess_pipeline_print.py "dataset/v_voice" 48000 2 "Mangio-RVC-Fork/logs/v_voice" 1
start preprocess
['Mangio-RVC-Fork/trainset_preprocess_pipeline_print.py', 'dataset/v_voice', '48000', '2', 'Mangio-RVC-Fork/logs/v_voice', '1']
thread:0: 100%|███████████████████████████████████| 5/5 [00:01<00:00,  2.87it/s]
end preprocess


In [35]:
%cd Mangio-RVC-Fork

/home/vishnu/hackathon/content


In [34]:
!pwd

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [37]:
#@title Feature Extraction

#pitch_extraction_algorithm = "harvest" #@param ["harvest", "crepe", "mangio-crepe"] {allow-input: false}
#crepe_hop_length = 128 #@param {type:"slider", min:1, max:512, step:1}
#pitch_guidance = True #@param {type:"boolean"}

gpuList = gpus.split("-")
cmd = "python3 Mangio-RVC-Fork/extract_f0_print.py \"%s/logs/%s\" %s %s %s" % (now_dir, experiment_name, cpu_threads, pitch_extraction_algorithm, crepe_hop_length)
print(cmd)
!$cmd

leng = len(gpus)

cmd = "python3 Mangio-RVC-Fork/extract_feature_print.py %s %s %s %s \"%s/logs/%s\" %s" % ("device", leng, 0, 0, now_dir, experiment_name, model_architecture)
print(cmd)
!$cmd


python3 Mangio-RVC-Fork/extract_f0_print.py "Mangio-RVC-Fork/logs/v_voice" 2 rmvpe 64
['Mangio-RVC-Fork/extract_f0_print.py', 'Mangio-RVC-Fork/logs/v_voice', '2', 'rmvpe', '64']
Using f0 method: rmvpe
thread:0, f0ing, Hop-Length:64: 100%|███████████| 18/18 [00:05<00:00,  3.49it/s]

thread:1, f0ing, Hop-Length:64: 100%|███████████| 18/18 [00:05<00:00,  3.45it/s]
python3 Mangio-RVC-Fork/extract_feature_print.py device 1 0 0 "Mangio-RVC-Fork/logs/v_voice" v2
['Mangio-RVC-Fork/extract_feature_print.py', 'device', '1', '0', '0', 'Mangio-RVC-Fork/logs/v_voice', 'v2']
Mangio-RVC-Fork/logs/v_voice
load model(s) from hubert_base.pt
2023-10-14 15:05:45 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/vishnu/hackathon/content
2023-10-14 15:05:45 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'nor

In [40]:
%cd ..


/home/vishnu/hackathon


In [47]:
%cd Mangio-RVC-Fork

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [38]:
#@title Save preprocessed dataset files to Google Drive
#Compress dataset folder
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip "{loc}"


  adding: Mangio-RVC-Fork/logs/v_voice/ (stored 0%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/ (stored 0%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/4_4.wav.npy (deflated 84%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/0_3.wav.npy (deflated 85%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/3_5.wav.npy (deflated 85%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/8_1.wav.npy (deflated 82%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/1_1.wav.npy (deflated 86%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/3_2.wav.npy (deflated 85%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/8_4.wav.npy (deflated 83%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/2_1.wav.npy (deflated 86%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/7_0.wav.npy (deflated 85%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/6_1.wav.npy (deflated 87%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/2_4.wav.npy (deflated 84%)
  adding: Mangio-RVC-Fork/logs/v_voice/2a_f0/3_6.wav.npy (deflated 87%)
  adding: Mangio-RVC-Fork/

In [51]:
%cd ..

/home/vishnu/hackathon


In [54]:

!cp content/Mangio-RVC-Fork/rvcLogs.zip "rvcLogs.zip"

In [57]:
%cd hackathon

/home/vishnu/hackathon


In [60]:
%cd hackathon/content/drive/MyDrive/rvcDisconnected/local_voice


/home/vishnu/hackathon/content/drive/MyDrive/rvcDisconnected/local_voice


In [39]:
!pwd

/home/vishnu/hackathon/content


In [40]:
#@title Load preprocessed dataset files from Google Drive (for resuming)
#@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
import os

BACK_UP_DATASET_PATH = "drive/MyDrive/rvcDisconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip "{BACK_UP_DATASET_PATH}/rvcLogs.zip" -d /

Dataset files already loaded, skipping.


In [65]:
%cd ..

/home/vishnu/hackathon


In [41]:
#@title Import Model from Drive to Notebook (for resuming)
import os

DATASET_PATH_DRIVE = "drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "Mangio-RVC-Fork/logs/" + experiment_name
#@markdown Input the model's Step Count here (the number located on your model's G and D files.) If you used `save_only_last_ckpt` during training, this number will be 2333333.
STEPCOUNT = 000 #@param {type:"integer"}

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

Copying model files...
cp: cannot stat 'drive/MyDrive/rvcDisconnected/v_voice/D_0.pth': No such file or directory


cp: cannot stat 'drive/MyDrive/rvcDisconnected/v_voice/G_0.pth': No such file or directory
cp: cannot stat 'drive/MyDrive/rvcDisconnected/v_voice/config.json': No such file or directory
Copying Tensorboard TFEVENT files...
All done. Welcome back!


In [71]:
%cd Mangio-RVC-Fork

/home/vishnu/hackathon/content


In [81]:
%cd ..

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [46]:
!pwd
print(now_dir)

/home/vishnu/hackathon/content


Mangio-RVC-Fork


In [47]:
%cd Mangio-RVC-Fork

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [48]:
now_dir = "/home/vishnu/hackathon/content/Mangio-RVC-Fork"
print(now_dir)

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [88]:
import os
import math
from random import shuffle

# I will not be adding an autosave feature. Do not ask.

#@title Training
save_frequency = 10 #@param {type:"integer"}
total_epochs = 5 #@param {type:"integer"}
batch_size = 20 #@param {type:"integer"}
save_only_latest_ckpt = True #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown The automatically calculated log interval is known to be very inaccurate and can cause delays between an epoch finishing and Tensorboard writes. If you would like, you can manually define a log interval here.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

assert save_frequency!=None, "You need to input something for save_frequency, silly."
assert save_frequency>0, "Save frequency must be more than 0."
if(save_frequency>50):print("...A save frequency of %s? A bit high, but... alright then."%save_frequency)
assert total_epochs!=None, "You need to input something for total_epochs, silly."
assert total_epochs>0, "Total epochs must be more than 0."
if(total_epochs>10000):print("...A total epoch count of of %s? This is going to overtrain, but... alright then."%total_epochs)
assert batch_size!=None, "You need to input something for batch_size, silly."
assert batch_size>0, "Batch size must be more than 0."
assert batch_size<=40, "Batch size must be less than 40. (I'd reccomend a value between 6 and 12 for Colab.)"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)

pretrainedD = "%sf0D%s.pth" % (pretrained_base, target_sample_rate)
pretrainedG = "%sf0G%s.pth" % (pretrained_base, target_sample_rate)

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print("That's a big dataset you got there. Log interval normalized to 200 steps from %s steps." % log_interval)
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python3 train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)
print('yes')

#Create mute filelist
exp_dir = "logs/%s" % ( experiment_name)
gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
feature_dir = (
  "%s/3_feature256" % (exp_dir)
  if model_architecture == "v1"
  else "%s/3_feature768" % (exp_dir)
)
f0_dir = "%s/2a_f0" % (exp_dir)
f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
      % (now_dir, target_sample_rate, now_dir, fea_dim, now_dir, now_dir, speaker_id)
  )
shuffle(opt)
with open("%s/filelist.txt" % exp_dir, "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


#%cd content/Mangio-RVC-Fork
#%cd Mangio-RVC-Fork
%load_ext tensorboard
#%tensorboard --logdir content/Mangio-RVC-Fork/logs
%tensorboard --logdir logs

#os.chdir('content/Mangio-RVC-Fork')
!$cmd


python3 train_nsf_sim_cache_sid_load_pretrain.py -e "v_voice" -sr 48k -f0 1 -bs 20 -g 0 -te 5 -se 10 -pg pretrained_v2/f0G48k.pth -pd pretrained_v2/f0D48k.pth -l 1 -c 0 -sw 1 -v v2 -li 3
yes
Mute filelist written. Best of luck training!
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 26795), started 0:11:25 ago. (Use '!kill 26795' to kill it.)

INFO:v_voice:{'train': {'log_interval': 3, 'seed': 1234, 'epochs': 20000, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 20, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 17280, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'max_wav_value': 32768.0, 'sampling_rate': 48000, 'filter_length': 2048, 'hop_length': 480, 'win_length': 2048, 'n_mel_channels': 128, 'mel_fmin': 0.0, 'mel_fmax': None, 'training_files': './logs/v_voice/filelist.txt'}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [12, 10, 2, 2], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [24, 20, 4, 4], 'use_spectral_norm': False, 'gin_channels': 256, 'spk_embed_dim': 109}, 'model_dir': './logs/v_voice', 'experiment_dir':

In [74]:
!pwd
from lib1.infer_pack import commons

/home/vishnu/hackathon/content/Mangio-RVC-Fork


In [60]:

import sys, os
sys.path.append('/home/vishnu/hackathon/content/Mangio-RVC-Fork')